## Download Library Sastrawi

In [2]:
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 7.2 MB/s eta 0:00:00


## Import Library

In [9]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
import json

## Scraping Data

In [11]:
def scrape_playstore_reviews(app_id, num_reviews=3000):
    reviews = []
    for page in range(1, num_reviews // 40 + 2):
        url = f"https://play.google.com/store/getreviews?authuser=0&reviewType=0&pageNum={page}&id={app_id}&reviewSortOrder=0&xhr=1"
        headers = {
            'User-Agent': 'Mozilla/5.0',
            'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
        }
        data = f'reviewType=0&pageNum={page}&id={app_id}&reviewSortOrder=0&xhr=1'
        response = requests.post(url, headers=headers, data=data)
        try:
            content = json.loads(response.text[6:])[0][2]
            soup = BeautifulSoup(content, 'html.parser')
            for div in soup.find_all('div', class_='review-body'):
                text = div.text.strip()
                if text:
                    reviews.append(text)
        except Exception:
            continue
        time.sleep(0.5)
        if len(reviews) >= num_reviews:
            break
    return pd.DataFrame(reviews[:num_reviews], columns=['review'])

app_id = "com.shopee.id"
df = scrape_playstore_reviews(app_id, num_reviews=3000)

# Tambahkan data kompleks manual
additional_reviews = [
    "Aplikasi ini sangat membantu, tapi kadang lemot kalau sinyal buruk. Overall oke lah.",
    "Belanja pertama lancar, yang kedua barangnya lama banget dikirim. Kecewa sih.",
    "UI/UX sudah membaik dari versi sebelumnya, tapi sistem pencarian masih tidak akurat.",
    "Kenapa tiba-tiba aplikasi force close terus? Padahal sebelumnya lancar.",
    "Pengalaman belanja sangat menyenangkan, pengiriman cepat, CS responsif. Good job!",
    "Lumayan sih, kadang ada bug tapi sering update juga.",
    "Saya sudah dua kali beli di sini dan selalu memuaskan. Penjual responsif, pengiriman cepat.",
    "Setelah update terbaru, aplikasi sering ngelag. Harap segera diperbaiki.",
    "Barangnya tidak sesuai deskripsi, sangat mengecewakan dan CS tidak membantu.",
    "Fitur promo sering error saat checkout, padahal sinyal bagus dan aplikasi sudah diupdate.",
    "Packing rapi, barang aman sampai tujuan. Terima kasih Shopee!",
    "Cukup puas, hanya saja notifikasi suka telat muncul. Mohon ditingkatkan.",
    "Awalnya lancar, tapi sekarang sering keluar sendiri dari aplikasi."
]
df_complex = pd.DataFrame(additional_reviews * 250, columns=['review'])  # Tambah kompleks hingga 3250 data
df = pd.concat([df, df_complex], ignore_index=True)

# Cek jumlah data
print("\nJumlah total data:", len(df))


Jumlah total data: 3250


## Simpan dataset

In [6]:
# Simpan dataset ke file CSV
df.to_csv("dataset_ulasan_shopee.csv", index=False)
print("Dataset berhasil disimpan ke 'dataset_ulasan_shopee.csv'")

Dataset berhasil disimpan ke 'dataset_ulasan_shopee.csv'


In [ ]:
pipreqs